# Automation for the analyst

A team of analysts prepares the monthly report on the prices of the product selected by the Board. Because they are aware you know Python, they asked you to automate the process. Talking to the team, you have set the following business conditions that enable process automation:

Three report parameters are available:
- **product_group_id**,
- **product**,
- **date**.

Assumptions for each parameter:

1. A parameter may have at most one value,
1. If the parameter is empty we return all records from the group,
1. We assume that the file is always prepared correctly (we want to practice report automation, not error handling).

Based on the above requirements:

1. load the  **config.xlsx** file using `openpyxl`,
1. prepare appropriate conditions to filter data from **product_cleaned.csv**,
1. based on the conditions filter the frame,
1. aggregate the data using a **pivot_table**:
   a) index-product, province,
   b) columns-dates,
   c) value-average product price,
   d) remember to remove 0,
6. save the file to the spreadsheet any way you want.

Hints:

1. You can save individual filtering conditions to variables and then use them all to filter `DataFrame`, the same as writing them all as before i.e. `df.loc[var1 & var2]`
1. If you decide to write with Pandas, be careful with the parameters passed to the function (what happens if you set `index=False`?). Link to the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html).

In [1]:
import pandas as pd
from openpyxl import load_workbook

In [3]:
config_file = "./../../01_Data/config.xlsx"
config_wb = load_workbook(config_file)
config_ws = config_wb.active

In [19]:
# Extract parameters from config.xlsx
product_group_id = config_ws.cell(row=2, column=2).value  # Assumes parameter in row 2, column A
product = config_ws.cell(row=3, column=2).value           # Assumes parameter in row 2, column B
date_filter = config_ws.cell(row=4, column=2).value       # Assumes parameter in row 2, column C

In [20]:
print(product_group_id)
print(product)
print(date_filter)

1
None
None


In [25]:
data = pd.read_csv(
  '../../01_Data/product_prices_cleaned.csv',
  sep=',',
  encoding='UTF-8',
  decimal='.'
)

In [27]:
# Prepare conditions for filtering
conditions = []
if product_group_id:
    conditions.append(data["product_group_id"] == product_group_id)
if product:
    conditions.append(data["product"] == product)
if date_filter:
    conditions.append(data["date"] == date_filter)

In [29]:
# Apply filters
if conditions:
    filtered_df = data.loc[pd.concat(conditions, axis=1).all(axis=1)]
else:
    filtered_df = data 

In [31]:
filtered_df.head()

,province,product_types,currency,product_group_id,product_line,value,date,product,year,month,quarter
5,HOLY CROSS,whole pickled cucumbers 0.9l - per 10pcs.,PLN,1,NaN,0.28,2010-04-01,whole pickled cucumbers 0.9l - per 10pcs.,2010,4,2
12,POMERANIA,30% tomato concentrate - per 1kg,PLN,1,NaN,7.46,1999-10-01,30% tomato concentrate - per 1kg,1999,10,4
15,POLAND,whole pickled cucumbers 0.9l - per 10pcs.,PLN,1,NaN,2.36,2004-12-01,whole pickled cucumbers 0.9l - per 10pcs.,2004,12,4
26,LOWER SILESIA,frozen carrot and pea mix - per 1kg,PLN,1,NaN,2.78,2005-07-01,frozen carrot and pea mix - per 1kg,2005,7,3
37,MASOVIA,"apple juice, boxed - per 1l",PLN,1,NaN,1.91,2007-08-01,"apple juice, boxed - per 1l",2007,8,3


In [33]:
# Pivot table
pivot_table = pd.pivot_table(
    filtered_df,
    values="value",
    index=["product", "province"],
    columns=["date"],
    aggfunc="mean"
)

In [36]:
pivot_table

date                                                         1999-01-01  \
product                                   province                        
30% tomato concentrate - per 1kg          GREATER POLAND           7.56   
                                          HOLY CROSS               3.81   
                                          KUYAVIA-POMERANIA        6.72   
                                          LESSER POLAND            6.23   
                                          LOWER SILESIA            6.01   
...                                                                 ...   
whole pickled cucumbers 0.9l - per 10pcs. SILESIA                  2.22   
                                          SUBCARPATHIA             2.46   
                                          WARMIA-MASURIA           2.11   
                                          WEST POMERANIA           2.15   
                                          ŁÓDŹ                     0.00   

date                                                         1999-02-01  \
product                                   province                        
30% tomato concentrate - per 1kg          GREATER POLAND           7.09   
                                          HOLY CROSS               2.67   
                                          KUYAVIA-POMERANIA        7.02   
                                          LESSER POLAND            4.35   
                                          LOWER SILESIA            5.52   
...                                                                 ...   
whole pickled cucumbers 0.9l - per 10pcs. SILESIA                  2.01   
                                          SUBCARPATHIA             2.50   
                                          WARMIA-MASURIA           0.38   
                                          WEST POMERANIA           0.50   
                                          ŁÓDŹ                     0.00   

date                                                         1999-03-01  \
product                                   province                        
30% tomato concentrate - per 1kg          GREATER POLAND           7.32   
                                          HOLY CROSS               0.06   
                                          KUYAVIA-POMERANIA        6.37   
                                          LESSER POLAND            3.41   
                                          LOWER SILESIA            6.22   
...                                                                 ...   
whole pickled cucumbers 0.9l - per 10pcs. SILESIA                  2.11   
                                          SUBCARPATHIA             2.53   
                                          WARMIA-MASURIA           1.03   
                                          WEST POMERANIA           1.45   
                                          ŁÓDŹ                     0.00   

date                                                         1999-04-01  \
product                                   province                        
30% tomato concentrate - per 1kg          GREATER POLAND           7.84   
                                          HOLY CROSS               0.57   
                                          KUYAVIA-POMERANIA        6.14   
                                          LESSER POLAND            1.07   
                                          LOWER SILESIA            6.04   
...                                                                 ...   
whole pickled cucumbers 0.9l - per 10pcs. SILESIA                  2.41   
                                          SUBCARPATHIA             2.26   
                                          WARMIA-MASURIA           1.81   
                                          WEST POMERANIA           2.02   
                                          ŁÓDŹ                     0.00   

date                                                         1999-05-01  \
product              

In [37]:
# Remove 0 values
pivot_table = pivot_table.replace(0, pd.NA).dropna(how="all")

In [38]:
pivot_table

date                                                        1999-01-01  \
product                                   province                       
30% tomato concentrate - per 1kg          GREATER POLAND          7.56   
                                          HOLY CROSS              3.81   
                                          KUYAVIA-POMERANIA       6.72   
                                          LESSER POLAND           6.23   
                                          LOWER SILESIA           6.01   
...                                                                ...   
whole pickled cucumbers 0.9l - per 10pcs. SILESIA                 2.22   
                                          SUBCARPATHIA            2.46   
                                          WARMIA-MASURIA          2.11   
                                          WEST POMERANIA          2.15   
                                          ŁÓDŹ                    <NA>   

date                                                        1999-02-01  \
product                                   province                       
30% tomato concentrate - per 1kg          GREATER POLAND          7.09   
                                          HOLY CROSS              2.67   
                                          KUYAVIA-POMERANIA       7.02   
                                          LESSER POLAND           4.35   
                                          LOWER SILESIA           5.52   
...                                                                ...   
whole pickled cucumbers 0.9l - per 10pcs. SILESIA                 2.01   
                                          SUBCARPATHIA             2.5   
                                          WARMIA-MASURIA          0.38   
                                          WEST POMERANIA           0.5   
                                          ŁÓDŹ                    <NA>   

date                                                        1999-03-01  \
product                                   province                       
30% tomato concentrate - per 1kg          GREATER POLAND          7.32   
                                          HOLY CROSS              0.06   
                                          KUYAVIA-POMERANIA       6.37   
                                          LESSER POLAND           3.41   
                                          LOWER SILESIA           6.22   
...                                                                ...   
whole pickled cucumbers 0.9l - per 10pcs. SILESIA                 2.11   
                                          SUBCARPATHIA            2.53   
                                          WARMIA-MASURIA          1.03   
                                          WEST POMERANIA          1.45   
                                          ŁÓDŹ                    <NA>   

date                                                        1999-04-01  \
product                                   province                       
30% tomato concentrate - per 1kg          GREATER POLAND          7.84   
                                          HOLY CROSS              0.57   
                                          KUYAVIA-POMERANIA       6.14   
                                          LESSER POLAND           1.07   
                                          LOWER SILESIA           6.04   
...                                                                ...   
whole pickled cucumbers 0.9l - per 10pcs. SILESIA                 2.41   
                                          SUBCARPATHIA            2.26   
                                          WARMIA-MASURIA          1.81   
                                          WEST POMERANIA          2.02   
                                          ŁÓDŹ                    <NA>   

date                                                        1999-05-01  \
product                                   province                       


In [39]:
# Save the output to Excel
output_file = "report.xlsx"
pivot_table.to_excel(output_file, sheet_name="Report")